## YH ansökningsomgång 2024

In [1]:
import pandas as pd
import duckdb

df = pd.read_excel(
    "../data/resultat_ansokning_program/2022_resultat-ansokningsomgang.xlsx", sheet_name="Tabell 3", 
)
df.head()

,Utbildningsområde,Utbildningsnamn,Beslut,Diarienummer,Län,Kommun,Flera kommuner,Antal kommuner,YH-poäng,Studieform,Studietakt %,Typ av examen,Utbildningsanordnare administrativ enhet,Huvudmannatyp,Sökta utbildningsomgångar,Beviljade utbildningsomgångar
0,Data/IT,.NET Cloud developer,Avslag,MYH 2022/5458,Stockholm,Stockholm,Nej,1,400,Bunden,100,Yrkeshögskoleexamen,IT-Högskolan Stockholm AB,Privat,3,0
1,Data/IT,.NET Developer,Avslag,MYH 2022/4695,Flera kommuner,Flera kommuner,Ja,3,400,Bunden,100,Yrkeshögskoleexamen,KYH AB,Privat,3,0
2,Data/IT,.NET Utvecklare,Beviljad,MYH 2022/4476,Gävleborg,Gävle,Nej,1,400,Bunden,100,Yrkeshögskoleexamen,Plushögskolan AB - Teknikhögskolan,Privat,5,3
3,Data/IT,.NET Utvecklare,Avslag,MYH 2022/4708,Flera kommuner,Flera kommuner,Ja,4,410,Distans,100,Yrkeshögskoleexamen,Plushögskolan AB - Teknikhögskolan Mitt,Privat,5,0
4,Data/IT,.NET-utvecklare,Avslag,MYH 2022/5529,Västra Götaland,Kungälv,Nej,1,400,Distans,100,Yrkeshögskoleexamen,Optimum Education AB,Privat,2,0


### Beviljandegrad, Beviljade utbildningar, Sökta utbildningar

In [2]:
decisions = df["Beslut"].value_counts()
approved, total = decisions["Beviljad"], decisions.sum()
approved, total 
beviljandegrad = approved / total *100

print(f"beviljandegrad = {beviljandegrad:.1f} %")
print(f"beviljade utbildningar = {approved}")
print(f"sökta utbildningar = {total}")

beviljandegrad = 34.8 %
beviljade utbildningar = 420
sökta utbildningar = 1207


### Beviljade platser, Ansökta platser

In [21]:
df_platser = pd.read_excel(
    "../data/resultat_ansokning_program/2022_resultat-ansokningsomgang.xlsx", sheet_name="Tabell 4", 
)
df_platser.head(2)

,Utbildningsområde,Utbildningsnamn,Beslut,Diarienummer,Län,Kommun,Flera kommuner,Antal kommuner,YH-poäng,Studieform,...,Utbildningsanordnare administrativ enhet,Huvudmannatyp,Sökta platser per utbildningsomgång,Sökta platser totalt,Beviljade platser utbildningsomgång 1,Beviljade platser utbildningsomgång 2,Beviljade platser utbildningsomgång 3,Beviljade platser utbildningsomgång 4,Beviljade platser utbildningsomgång 5,Beviljade platser totalt
0,Data/IT,.NET Cloud developer,Avslag,MYH 2022/5458,Stockholm,Stockholm,Nej,1,400,Bunden,...,IT-Högskolan Stockholm AB,Privat,35,105,0.0,0.0,0,0,0,0.0
1,Data/IT,.NET Developer,Avslag,MYH 2022/4695,Blekinge,Ronneby,Ja,3,400,Bunden,...,KYH AB,Privat,11,33,0.0,0.0,0,0,0,0.0


In [22]:
total_approved = int(df_platser["Beviljade platser totalt"].sum())
total_applied = df_platser["Sökta platser totalt"].sum()

print(f"beviljade platser = {total_approved}")
print(f"ansökta platser = {total_applied}") # 166263 IS WRONG BECAUSE 131176 + 35087 = 166263


beviljade platser = 35087
ansökta platser = 131176


### Sökta bundna utbildningar, Sökta distansutbildningar

In [19]:
result = duckdb.query("""--sql
    WITH unique_forms AS (
        SELECT
            Diarienummer,
            COUNT(DISTINCT TRIM(LOWER(Studieform))) AS form_count
        FROM df
        GROUP BY Diarienummer
    ),
    filtered_programs AS (
        SELECT
            t.Diarienummer,
            TRIM(LOWER(t.Studieform)) AS Studieform
        FROM df t
        JOIN unique_forms u ON t.Diarienummer = u.Diarienummer
        WHERE u.form_count = 1
    )
    SELECT
        Studieform,
        COUNT(DISTINCT Diarienummer) AS antal_program
    FROM filtered_programs
    GROUP BY Studieform
""").to_df()

# To print count where Studieform == 'bunden'
bunden_count = result.loc[result['Studieform'] == 'bunden', 'antal_program'].sum()

print(f"sökta bundna utbildningar = {bunden_count}")

# To print count where Studieform == 'bunden'
distans_count = result.loc[result['Studieform'] == 'distans', 'antal_program'].sum()

print(f"Sökta distansutbildningar = {distans_count}")

sökta bundna utbildningar = 625
Sökta distansutbildningar = 582


In [6]:
df_regions = duckdb.query(
    """--sql
    SELECT 
        län,
        CAST(COUNT_IF(beslut = 'Beviljad') AS INTEGER) AS Beviljade
    FROM df
    WHERE län != 'Flera kommuner'
    GROUP BY län
    ORDER BY beviljade  DESC, län ASC

""").df()

df_regions

,Län,Beviljade
0,Stockholm,79
1,Västra Götaland,77
2,Skåne,51
3,Jönköping,21
4,Östergötland,16
5,Norrbotten,14
6,Västerbotten,14
7,Västernorrland,14
8,Örebro,9
9,Jämtland,8
